In [ ]:
import os  # <--- Add this line
from langgraph.graph import StateGraph ,START, END
from typing import TypedDict
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

In [2]:
load_dotenv()
ollama_url = os.getenv("OLLAMA_BASE_URL")
model = ChatOllama(
    model="llama3.2:latest", 
    base_url=ollama_url,  # This is the crucial part!
    temperature=0
)

In [3]:
class blogState(TypedDict):
    topic:str
    outline:str  
    blog:str

In [4]:
def generate_outline(state:blogState)->blogState:
    #fetch title
    topic=state['topic']    

    #form a prompt
    prompt=f'generate outline for topic {topic}'

    #call lmm for outline
    outline=model.invoke(prompt).content

    #updatestate
    state['outline']=outline

    return state

In [5]:
def generate_blog(state:blogState)->blogState:
    #fetch title + outline
    topic=state['topic']  
    outline=state['blog']  

    #form a blog prompt
    prompt=f'generate detail blog for given topic and outline {topic , outline} '

    #call llm for blog
    blog=model.invoke(prompt).content

    #updating blog state
    state['blog']=blog
    
    return state

In [6]:
#3.define graph(pass state)
graph=StateGraph(blogState)

#4.make node of function
graph.add_node('generate_outline'  , generate_outline)
graph.add_node('generate_blog'  , generate_blog)

#5.add edges between nodes
graph.add_edge(START,'generate_outline')
graph.add_edge('generate_blog','generate_outline')
graph.add_edge('generate_blog',END)

#6.complie workflow
workflow=graph.compile()

In [7]:
#execute
inital_state={'topic':'Artificial intellegence'}
final_state = workflow.invoke(inital_state)
print(final_state)

{'topic': 'Artificial intellegence', 'outline': 'Here is a comprehensive outline on the topic of Artificial Intelligence (AI):\n\n**I. Introduction**\n\n* Definition of Artificial Intelligence\n* Brief history of AI development\n* Importance and relevance of AI in modern society\n\n**II. Types of Artificial Intelligence**\n\n* Narrow or Weak AI: designed to perform a specific task\n* General or Strong AI: aims to replicate human intelligence and reasoning\n* Superintelligence: significantly surpasses human intelligence\n\n**III. Applications of Artificial Intelligence**\n\n* Virtual Assistants (e.g., Siri, Alexa)\n* Image Recognition and Computer Vision\n* Natural Language Processing (NLP) and Chatbots\n* Predictive Analytics and Machine Learning\n* Robotics and Autonomous Vehicles\n* Healthcare and Medical Diagnosis\n* Finance and Trading\n\n**IV. AI Techniques and Algorithms**\n\n* Supervised Learning: training models on labeled data\n* Unsupervised Learning: discovering patterns in 